In [14]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

In [17]:
import sys
import argparse

sys.path.append('../../../')
from el_evaluation import *
from utils import MyBatchSampler
from torch.utils.data import DataLoader, TensorDataset
from utils import QuestionSampler
import copy
import torch
import random

random.seed(103)

device = torch.device("cpu")

parser = argparse.ArgumentParser(description="main training script for training lnn entity linking models")
# parser.add_argument("--gmt_data", type=str, default="./data/train_sorted.csv", help="train csv")
parser.add_argument("--train_data", type=str, default="./data/train_filtered_sorted.csv", help="train csv")
parser.add_argument("--test_data", type=str, default="./data/test_sorted.csv", help="test csv")
parser.add_argument("--checkpoint_name", type=str, default="checkpoint/best_model.pt", help="checkpoint path")
parser.add_argument("--output_file_name", type=str, default="output/purename_nway_alpha09.txt", help="checkpoint path")
parser.add_argument("--model_name", type=str, default="purename", help="which model we choose")
# args for dividing the corpus
parser.add_argument('--alpha', type=float, default=0.9, help='alpha for LNN')
parser.add_argument('--num_epoch', type=int, default=100, help='training epochs for LNN')
parser.add_argument("--use_binary", action="store_true", help="default is to use binary`, otherwise use stem")
parser.add_argument('--margin', type=float, default=0.15, help='margin for MarginRankingLoss')
parser.add_argument('--learning_rate', type=float, default=0.001, help='learning rate')
parser.add_argument("-f")
args = parser.parse_args()

# from RuleLNN_nway_sigmoid_vec import *

In [18]:
kf = KFold(n_splits=5)

In [19]:
all_instances = list(range(100))


In [20]:
for train_index, test_index in kf.split(all_instances):
    pass

In [25]:
df_train_val = pd.read_csv(args.train_data)
print(df_train_val.shape)
df_test = pd.read_csv(args.test_data)
print(df_test.shape)

(22462, 8)
(17808, 7)


In [23]:
df_data = pd.concat([df_train_val, df_test])

In [26]:
df_data['QuestionMention'] = df_data.Question + df_data.Mention

In [28]:
only_negative_pairs = []
only_one_instance_no_negatives = []
for unique_qm_pair in set(df_data.QuestionMention):
    df_ = df_data[df_data.QuestionMention == unique_qm_pair]
    if df_.Label.sum() == 1 and df_.shape[0] == 1:
        print(unique_qm_pair)
        only_one_instance_no_negatives.append(unique_qm_pair)
    if df_.Label.sum() == 0:
        print(unique_qm_pair)
        only_negative_pairs.append(unique_qm_pair)

How high is the Yokohama Marine Tower?yokohama marine tower
In which city does the Chile Route 68 end?chile
In which country is the Limerick Lake?limerick lake
Which country does the creator of Miffy come from?miffy
In which city did John F. Kennedy die?john f. kennedy
Who was the successor of John F. Kennedy?john f. kennedy
Do Prince Harry and Prince William have the same parents?harry
How much did the Lego Movie cost?lego movie
Is Michelle Obama the wife of Barack Obama?michelle obama
Who wrote the Game of Thrones theme?game of thrones
In which city are the headquarters of the United Nations?united nations
What country is Sitecore from?sitecore
Who was John F. Kennedy's vice president?john f. kennedy
Which books by Kerouac were published by Viking Press?viking press
With how many countries Iran has borders?iran
Is there a video game called Battle Chess?battle chess
How tall is Claudia Schiffer?claudia schiffer
Who was the wife of U.S. president Lincoln?u.s.
Does Breaking Bad have mor

In [30]:
print(df_data.shape)
df_filter_negatives = df_data[~df_data.QuestionMention.isin(only_negative_pairs)]
print(df_filter_negatives.shape)

(40270, 8)
(37376, 8)


In [32]:
print(df_filter_negatives.shape)
df_filter_negatives1 = df_filter_negatives[~df_filter_negatives.QuestionMention.isin(only_one_instance_no_negatives)]
print(df_filter_negatives1.shape)

(37376, 8)
(37365, 8)


In [33]:
df_filter_negatives1.to_csv("data/data_filtered_sorted.csv", index=False)

NameError: name 'df_train' is not defined

In [45]:
df_data = pd.read_csv("data/data_filtered_sorted.csv")
questions_idxes = list(range(len(df_data.Question.unique())))

In [58]:
kf = KFold(n_splits=5)
question_list = np.array(df_data.Question.unique())
FOLDNUM = 1
for train_index, test_index in kf.split(questions_idxes):
    print("=======FOLD-{}========".format(FOLDNUM))
    FOLDNUM += 1
    print(train_index.shape)
#     print(question_list[train_index])
    train_ques_set = question_list[train_index]
    test_ques_test = question_list[test_index]

    df_train = df_data[df_data.Question.isin(train_ques_set)]
    df_test = df_data[df_data.Question.isin(test_ques_test)]
    
    # train
    features_train = np.array(
        [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_train.Features.values])
    x_train = torch.from_numpy(features_train).float()
    y_train = torch.from_numpy(df_train.Label.values).float().reshape(-1, 1)
    m_labels_train = df_train.Mention_label.values
    ques_train = df_train.Question.values

    # test
    features_test = np.array(
        [np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_test.Features.values])
    x_test = torch.from_numpy(features_test).float()
    y_test = torch.from_numpy(df_test.Label.values).float().reshape(-1, 1)
    m_labels_test = df_test.Mention_label.values
    ques_test = df_test.Question.values

    # aggregate the data into train, val, and test
    print("Number of mention-entity pair instances")
    train_data = (x_train, y_train, m_labels_train, ques_train)
    print("train:", x_train.shape, y_train.shape, m_labels_train.shape, ques_train.shape)
    test_data = (x_test, y_test, m_labels_test, ques_test)
    print("test:", x_test.shape, y_test.shape, m_labels_test.shape, ques_test.shape)

    # check class distribution
    print("class distribution")
    print("y_train sum", sum(y_train), sum(y_train) / len(y_train))
    print("y_test sum", sum(y_test), sum(y_test) / len(y_test))

=======FOLD-1========
(242,)
Number of mention-entity pair instances
train: torch.Size([27197, 6]) torch.Size([27197, 1]) (27197,) (27197,)
test: torch.Size([10168, 6]) torch.Size([10168, 1]) (10168,) (10168,)
class distribution
y_train sum tensor([276.]) tensor([0.0101])
y_test sum tensor([79.]) tensor([0.0078])
=======FOLD-2========
(242,)
Number of mention-entity pair instances
train: torch.Size([30594, 6]) torch.Size([30594, 1]) (30594,) (30594,)
test: torch.Size([6771, 6]) torch.Size([6771, 1]) (6771,) (6771,)
class distribution
y_train sum tensor([292.]) tensor([0.0095])
y_test sum tensor([63.]) tensor([0.0093])
=======FOLD-3========
(242,)
Number of mention-entity pair instances
train: torch.Size([31597, 6]) torch.Size([31597, 1]) (31597,) (31597,)
test: torch.Size([5768, 6]) torch.Size([5768, 1]) (5768,) (5768,)
class distribution
y_train sum tensor([288.]) tensor([0.0091])
y_test sum tensor([67.]) tensor([0.0116])
=======FOLD-4========
(243,)
Number of mention-entity pair inst